<a href="https://colab.research.google.com/github/francescobellina/Repo_laboratorio_BiophotonicsI/blob/main/Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PATRICLE TRACKING AND DIFFUSION

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print("ciao")

ciao


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm

file_path = '/content/drive/MyDrive/Experiment_spots_002.csv'

df = pd.read_csv(file_path, skiprows=3)
print("Nomi colonne effettive:", list(df.columns))

# Estrai le colonne delle coordinate (micron) in base all'ordine delle colonne
x_data = df['(micron)'].astype(float).values
y_data = df['(micron).1'].astype(float).values

# ---- Stampa ordinata delle prime 20 coordinate x/y ----
first20_df = pd.DataFrame({'X (micron)': x_data[:20], 'Y (micron)': y_data[:20]})
print("\nPrime 20 coordinate (x,y):")
print(first20_df)

# Se hai più tracce, ricostruisci traiettorie (qui presupponiamo Track ID in 'Unnamed: 1')
tracks = df['Unnamed: 1'].astype(str).values
frames = df['Unnamed: 9'] if 'Unnamed: 9' in df.columns else np.arange(len(x_data))

unique_tracks = np.unique(tracks)
num_frames = len(np.unique(frames))
x_trajs = np.zeros((len(unique_tracks), num_frames))
y_trajs = np.zeros((len(unique_tracks), num_frames))
for i, track in enumerate(unique_tracks):
    mask = tracks == track
    sort_idx = np.argsort(frames[mask])
    x_trajs[i, :] = x_data[mask][sort_idx]
    y_trajs[i, :] = y_data[mask][sort_idx]

# --- MSD analysis ---
def eaMSD(x_trajs, y_trajs, n):
    return np.nanmean((x_trajs[:, n] - x_trajs[:, 0])**2 + (y_trajs[:, n] - y_trajs[:, 0])**2)
def taMSD(x, y, n):
    disp = (x[n:] - x[:-n])**2 + (y[n:] - y[:-n])**2
    return np.nanmean(disp)
def msd_vs_time(x_trajs, y_trajs, max_lag):
    ea = [eaMSD(x_trajs, y_trajs, n) for n in range(1, max_lag)]
    ta = np.mean([[taMSD(x, y, n) for n in range(1, max_lag)] for x, y in zip(x_trajs, y_trajs)], axis=0)
    return ea, ta

M, N = x_trajs.shape
max_lag = min(N//2, N-1)
dt = 0.1
time = np.arange(N) * dt

ea, ta = msd_vs_time(x_trajs, y_trajs, max_lag)
lags = np.arange(1, max_lag) * dt

plt.figure()
plt.plot(lags, ea, label='Ensemble MSD')
plt.plot(lags, ta, label='Time-averaged MSD')
if len(ea) > 1:
    D_estimate = ea[1] / (4 * dt)
    plt.plot(lags, 4 * D_estimate * lags, '--', label=f'4Dt (theory, D~{D_estimate:.2f})')
plt.xlabel(r't (s)')
plt.ylabel(r'MSD ($\mu m^2$)')
plt.legend()
plt.title('Mean Square Displacement')
plt.show()

plt.figure()
for t_idx, t_col in zip([10, 20, 40, 80], ['b', 'r', 'g', 'c']):
    if t_idx < N:
        x_now = x_trajs[:, t_idx]
        mu, std = np.nanmean(x_now), np.nanstd(x_now)
        x_vals = np.linspace(mu - 4*std, mu + 4*std, 100)
        plt.plot(x_vals, norm.pdf(x_vals, mu, std), label=f't={time[t_idx]:.1f}s')
plt.xlabel(r'x ($\mu m$)')
plt.ylabel('p(x,t)')
plt.title('Distribution of x at different times')
plt.legend()
plt.show()


Nomi colonne effettive: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', '(quality)', '(micron)', '(micron).1', '(micron).2', '(sec)', 'Unnamed: 8', '(micron).3', 'Unnamed: 10', 'Unnamed: 11', '(counts)', '(counts).1', '(counts).2', '(counts).3', '(counts).4', '(counts).5', 'Unnamed: 18', 'Unnamed: 19']

Prime 20 coordinate (x,y):
    X (micron)  Y (micron)
0    10.469439   18.039649
1    10.308371   17.798047
2    10.066769   18.120183
3    10.630508   17.717513
4    10.469439   18.039649
5    10.952644   17.234308
6    10.711042   17.959115
7    10.791576   17.475910
8    11.194247   17.234308
9    11.435849   17.073240
10   11.435849   16.912171
11   11.516383   16.912171
12   12.080122   16.106830
13   12.160656   15.945761
14   12.241191   15.784693
15   12.482793   14.818283
16   12.402259   15.220954
17   12.965998   14.657215
18   13.288135   14.415613
19   13.288135   14.496147
